In [1]:
import os, csv
import numpy as np
from PIL import Image
import re

from tqdm import tqdm_notebook
tqdm_notebook()
from sklearn.metrics import jaccard_similarity_score

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [8]:
import time


In [9]:
for index in tqdm_notebook(range(1000)):
    time.sleep(0.1)

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
for index in tqdm(range(1000)):
    time.sleep(0.1)

In [2]:
def compute_mean_iou(pred, label):

    unique_labels = np.unique(label)
    num_unique_labels = len(unique_labels);

    I = np.zeros(num_unique_labels)
    U = np.zeros(num_unique_labels)

    for index, val in enumerate(unique_labels):
        pred_i = pred == val
        label_i = label == val

        I[index] = float(np.sum(np.logical_and(label_i, pred_i)))
        U[index] = float(np.sum(np.logical_or(label_i, pred_i)))


    mean_iou = np.mean(I / U)
    return mean_iou

In [3]:

def get_label_info(csv_path):
    """
    Retrieve the class names and label values for the selected dataset.
    Must be in CSV format!
    # Arguments
        csv_path: The file path of the class dictionairy
        
    # Returns
        Two lists: one for the class names and the other for the label values
    """
    filename, file_extension = os.path.splitext(csv_path)
    if not file_extension == ".csv":
        return ValueError("File is not a CSV!")

    class_names = []
    label_values = []
    with open(csv_path, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        header = next(file_reader)
        for row in file_reader:
            print (row)
            class_names.append(row[0])
            #print (class_names)
            label_values.append([int(row[1]), int(row[2]), int(row[3])])
    print (class_names)
    print(label_values)
        #print(class_dict)
    return class_names, label_values

In [4]:
def compute_class_accuracies(pred, label, num_classes):
    total = []
    for val in range(num_classes):
        total.append((label == val).sum())

    count = [0.0] * num_classes
    for i in range(len(label)):
        if pred[i] == label[i]:
            count[int(pred[i])] = count[int(pred[i])] + 1.0

    # If there are no pixels from a certain class in the GT, 
    # it returns NAN because of divide by zero
    # Replace the nans with a 1.0.
    accuracies = []
    for i in range(len(total)):
        if total[i] == 0:
            accuracies.append(1.0)
            
        else:
            accuracies.append(count[i] / total[i])
    print (accuracies)

    return accuracies


In [5]:

# Get the names of the classes so we can record the evaluation results
class_names_list, label_values = get_label_info("/home/schind/Dokumente/synthetic-dataset/iou/class_dict.csv")
class_names_string = ""
for class_name in class_names_list:
    if not class_name == class_names_list[-1]:
        class_names_string = class_names_string + class_name + ", "
    else:
        class_names_string = class_names_string + class_name
num_classes = len(label_values)

['Terrain', '0', '0', '0']
['Person', '184', '145', '182']
['Bike', '135', '169', '180']
['Construction', '65', '54', '217']
['Sidewalk', '211', '80', '208']
['Road', '142', '1', '246']
['Vegitation', '55', '181', '57']
['Car', '134', '57', '119']
['Sky', '90', '162', '242']
['Sign', '212', '51', '60']
['Misc', '81', '13', '36']
['Terrain', 'Person', 'Bike', 'Construction', 'Sidewalk', 'Road', 'Vegitation', 'Car', 'Sky', 'Sign', 'Misc']
[[0, 0, 0], [184, 145, 182], [135, 169, 180], [65, 54, 217], [211, 80, 208], [142, 1, 246], [55, 181, 57], [134, 57, 119], [90, 162, 242], [212, 51, 60], [81, 13, 36]]


In [7]:
#target = "G:/result_aeroscapes/vis/raw_segmentation_results/b'002000_001'.png"
#prediction = "G:/aeroscapes/SegmentationClass/002000_001.png"

pathImages = "/home/schind/Dokumente/Neighborhood_dataset/SegmentationRaw"

class_scores = []       
iou_list = []
i = 0
train = []
iterS = 0

pathRaw = "/home/schind/Dokumente/Neighborhood_dataset/vis_xception71/raw_segmentation_results"



path = os.listdir(pathImages)
pathR = os.listdir(pathRaw)
lengthR = len(pathR)

#for i in tqdm_notebook(range(len(pathR))):
#    i += 1
for k in pathR:

    knew= re.sub("b'","",k)
    knew = re.sub("'", "",knew)
    print(knew)
    original_im = Image.open(os.path.join(pathImages, knew))

    seg_raw = Image.open(os.path.join(pathRaw,k))

    i += 1
    print('running deeplab on image %s...' % k)



    target = np.asarray(seg_raw)
    prediction = np.asarray(original_im)

    #print (prediction)
    #print(target)

    #print (raw_imgs_loc)

    flat_pred = prediction.flatten()
    flat_label = target.flatten()
    iou =compute_class_accuracies(flat_pred, flat_label, num_classes)
    #iou = jaccard_similarity_score(flat_pred, flat_label)
    #print(iou)
    class_scores.append(iou)

    print (str(i) + " von 781")
class_avg_scores = np.mean(class_scores, axis=0)
print("Average per class validation accuracies for epoch" )
for index, item in enumerate(class_avg_scores):
    print("%s = %f" % (class_names_list[index], item))


Neighborhood_test_775_0_x-67_y2_z-22_r0_p-37_y0.png
running deeplab on image b'Neighborhood_test_775_0_x-67_y2_z-22_r0_p-37_y0'.png...
[0.9177717890548476, 1.0, 1.0, 0.9345027273013905, 1.0, 0.9748503990175019, 0.9083454354798373, 0.8443825168671164, 1.0, 1.0, 1.0]
1 von 781
Neighborhood_test_483_3_x-118_y-32_z-22_r0_p0_y0.png
running deeplab on image b'Neighborhood_test_483_3_x-118_y-32_z-22_r0_p0_y0'.png...
[0.723486187603173, 1.0, 1.0, 0.9708070185854478, 1.0, 0.9916615286997851, 0.03125, 0.9560274828232355, 1.0, 1.0, 1.0]
2 von 781
Neighborhood_test_987_4_x-97_y13_z-13_r0_p-47_y0.png
running deeplab on image b'Neighborhood_test_987_4_x-97_y13_z-13_r0_p-47_y0'.png...
[0.9092709583218886, 1.0, 1.0, 0.995373337253808, 1.0, 0.9084070339653808, 0.9312571942267056, 0.9443585780525502, 1.0, 1.0, 1.0]
3 von 781
Neighborhood_test_230_0_x-3_y5_z-29_r0_p-8_y0.png
running deeplab on image b'Neighborhood_test_230_0_x-3_y5_z-29_r0_p-8_y0'.png...
[0.9241420654764516, 1.0, 1.0, 0.8855810414788098,

[0.9092922253984222, 1.0, 1.0, 0.873651223556169, 1.0, 0.9582035027398732, 0.9229482920856621, 0.9373589164785553, 0.9838174533617128, 1.0, 1.0]
30 von 781
Neighborhood_test_452_0_x-12_y14_z-33_r0_p-8_y0.png
running deeplab on image b'Neighborhood_test_452_0_x-12_y14_z-33_r0_p-8_y0'.png...
[0.9247816541291286, 1.0, 1.0, 0.850437823548849, 1.0, 0.9146817798881826, 0.9255990965206764, 0.6692913385826772, 0.9919242100058567, 1.0, 1.0]
31 von 781
Neighborhood_test_815_3_x-88_y-27_z-33_r0_p0_y0.png
running deeplab on image b'Neighborhood_test_815_3_x-88_y-27_z-33_r0_p0_y0'.png...
[0.8100669936595286, 1.0, 1.0, 0.9491392384210245, 1.0, 0.9960236584443417, 0.9564824608409481, 0.835236541598695, 1.0, 1.0, 1.0]
32 von 781
Neighborhood_test_435_3_x-25_y-21_z-26_r0_p0_y0.png
running deeplab on image b'Neighborhood_test_435_3_x-25_y-21_z-26_r0_p0_y0'.png...
[0.9729117941236191, 1.0, 1.0, 0.960675979547136, 1.0, 0.9801939671067188, 0.8547223021582734, 0.9019041874892297, 1.0, 1.0, 1.0]
33 von 781
N

[0.8926965686492108, 1.0, 1.0, 0.9557926609473001, 1.0, 0.9554235178422837, 0.9297228134790615, 0.810958904109589, 1.0, 1.0, 1.0]
60 von 781
Neighborhood_test_620_0_x-1_y-34_z-18_r0_p-28_y0.png
running deeplab on image b'Neighborhood_test_620_0_x-1_y-34_z-18_r0_p-28_y0'.png...
[0.8709774353931049, 1.0, 1.0, 0.8820860477022756, 1.0, 0.9373294185758002, 0.896008480861755, 0.8488964346349746, 1.0, 1.0, 1.0]
61 von 781
Neighborhood_test_474_4_x-21_y-19_z-34_r0_p-3_y0.png
running deeplab on image b'Neighborhood_test_474_4_x-21_y-19_z-34_r0_p-3_y0'.png...
[0.8155368584262489, 1.0, 1.0, 0.916700137307466, 1.0, 0.8842262290828902, 0.9261866986041745, 0.6223175965665236, 0.9920537485315924, 1.0, 1.0]
62 von 781
Neighborhood_test_271_4_x-53_y46_z-33_r0_p-59_y0.png
running deeplab on image b'Neighborhood_test_271_4_x-53_y46_z-33_r0_p-59_y0'.png...
[0.9601577817649029, 1.0, 1.0, 0.8431091269330726, 1.0, 0.7517301038062284, 0.8399035593557367, 0.0, 1.0, 1.0, 1.0]
63 von 781
Neighborhood_test_868_3_

[0.9550213658115008, 1.0, 1.0, 0.9611365797751713, 1.0, 0.9609945739488921, 0.883510161602729, 0.8869637728157139, 1.0, 1.0, 1.0]
89 von 781
Neighborhood_test_766_0_x-9_y-60_z-32_r0_p-4_y0.png
running deeplab on image b'Neighborhood_test_766_0_x-9_y-60_z-32_r0_p-4_y0'.png...
[0.8982142857142857, 1.0, 1.0, 0.8715356988266124, 1.0, 0.8601717728496346, 0.9412683336149781, 0.6003210272873194, 0.9929839074120882, 1.0, 1.0]
90 von 781
Neighborhood_test_804_4_x-85_y-23_z-40_r0_p-27_y0.png
running deeplab on image b'Neighborhood_test_804_4_x-85_y-23_z-40_r0_p-27_y0'.png...
[0.8464419717111286, 1.0, 1.0, 0.9173606521260189, 1.0, 0.9241318711611519, 0.9317066223005693, 0.7930825242718447, 0.9638668206419657, 1.0, 1.0]
91 von 781
Neighborhood_test_492_0_x-121_y19_z-40_r0_p-35_y0.png
running deeplab on image b'Neighborhood_test_492_0_x-121_y19_z-40_r0_p-35_y0'.png...
[0.8600952450196306, 1.0, 1.0, 0.9080912087104127, 1.0, 0.9543744069109193, 0.8670244947762279, 0.7540425531914894, 1.0, 1.0, 1.0]
9

[0.9700955426260761, 1.0, 1.0, 0.9850349703754144, 1.0, 0.9650833902272505, 0.9044549076421586, 0.9459868142502873, 1.0, 1.0, 1.0]
118 von 781
Neighborhood_test_693_3_x-34_y1_z-21_r0_p0_y0.png
running deeplab on image b'Neighborhood_test_693_3_x-34_y1_z-21_r0_p0_y0'.png...
[0.9514154138427193, 1.0, 1.0, 0.9749217198170504, 1.0, 0.9822965065571431, 0.9029911504424779, 0.8958647511707278, 1.0, 1.0, 1.0]
119 von 781
Neighborhood_test_687_4_x-92_y-13_z-30_r0_p-5_y0.png
running deeplab on image b'Neighborhood_test_687_4_x-92_y-13_z-30_r0_p-5_y0'.png...
[0.7665664366929369, 1.0, 1.0, 0.8820501378551963, 1.0, 0.9899097185342538, 0.9208780769636736, 0.9846723044397463, 0.9903741634340765, 1.0, 1.0]
120 von 781
Neighborhood_test_327_4_x-110_y-51_z-39_r0_p-54_y0.png
running deeplab on image b'Neighborhood_test_327_4_x-110_y-51_z-39_r0_p-54_y0'.png...
[0.9356275878866607, 1.0, 1.0, 0.9418817066074884, 1.0, 0.9628105260808428, 0.9216117966309969, 0.8440489432703003, 1.0, 1.0, 1.0]
121 von 781
Neig

...

[0.9186691026595786, 1.0, 1.0, 0.884309806847204, 1.0, 0.9474707861048671, 0.8589177150964915, 0.746234067207416, 1.0, 1.0, 1.0]
619 von 781
Neighborhood_test_229_0_x-6_y34_z-37_r0_p-7_y0.png
running deeplab on image b'Neighborhood_test_229_0_x-6_y34_z-37_r0_p-7_y0'.png...
[0.8867984176907363, 1.0, 1.0, 0.8215438906043229, 1.0, 0.9008259638677291, 0.9218712122555889, 0.7851140456182473, 0.9923058747484405, 1.0, 1.0]
620 von 781
Neighborhood_test_61_0_x-49_y22_z-39_r0_p-27_y0.png
running deeplab on image b'Neighborhood_test_61_0_x-49_y22_z-39_r0_p-27_y0'.png...
[0.8514666401511385, 1.0, 1.0, 0.8786660830116598, 1.0, 0.9010511617082347, 0.901846922382354, 0.8278065440123389, 0.9682450138302519, 1.0, 1.0]
621 von 781
Neighborhood_test_96_3_x-8_y4_z-30_r0_p0_y0.png
running deeplab on image b'Neighborhood_test_96_3_x-8_y4_z-30_r0_p0_y0'.png...
[0.8926994931855, 1.0, 1.0, 0.9391918000447418, 1.0, 0.9746244516094931, 0.8890100549393594, 0.8348163454645131, 1.0, 1.0, 1.0]
622 von 781
Neighborh

[0.8567133788477662, 1.0, 1.0, 0.8363357195947267, 1.0, 0.8891977346165874, 0.9050732481778163, 0.6518518518518519, 0.9914585015837847, 1.0, 1.0]
648 von 781
Neighborhood_test_411_0_x-39_y-22_z-11_r0_p-2_y0.png
running deeplab on image b'Neighborhood_test_411_0_x-39_y-22_z-11_r0_p-2_y0'.png...
[0.7176237366351434, 1.0, 1.0, 0.9452835686222907, 1.0, 0.9045248758640834, 0.9170593066482234, 0.6897119341563785, 0.9837359354141504, 1.0, 1.0]
649 von 781
Neighborhood_test_806_4_x-56_y-28_z-39_r0_p-18_y0.png
running deeplab on image b'Neighborhood_test_806_4_x-56_y-28_z-39_r0_p-18_y0'.png...
[0.7950205555769009, 1.0, 1.0, 0.8617217153378525, 1.0, 0.9075568470250495, 0.9268752173134266, 0.6833855799373041, 0.987523270463905, 1.0, 1.0]
650 von 781
Neighborhood_test_389_0_x-122_y35_z-11_r0_p-13_y0.png
running deeplab on image b'Neighborhood_test_389_0_x-122_y35_z-11_r0_p-13_y0'.png...
[0.9114461371969138, 1.0, 1.0, 0.9300088427583607, 1.0, 0.8945307035949084, 0.880920379370767, 1.0, 0.9457263023

[0.7704254348997555, 1.0, 1.0, 0.8745286464918935, 1.0, 0.8145474996484979, 0.8938364166139806, 0.6666666666666666, 0.9908731456431905, 1.0, 1.0]
677 von 781
Neighborhood_test_394_0_x-84_y47_z-32_r0_p-48_y0.png
running deeplab on image b'Neighborhood_test_394_0_x-84_y47_z-32_r0_p-48_y0'.png...
[0.9395951509666076, 1.0, 1.0, 0.9351618194435682, 1.0, 0.900383408853259, 0.8908110972272749, 0.8734971249346576, 1.0, 1.0, 1.0]
678 von 781
Neighborhood_test_68_4_x-78_y23_z-26_r0_p-26_y0.png
running deeplab on image b'Neighborhood_test_68_4_x-78_y23_z-26_r0_p-26_y0'.png...
[0.8688218087512439, 1.0, 1.0, 0.8549124036440084, 1.0, 0.922748896941014, 0.901783353029185, 0.8495090634441088, 0.9071679145505, 1.0, 1.0]
679 von 781
Neighborhood_test_838_0_x8_y43_z-25_r0_p-58_y0.png
running deeplab on image b'Neighborhood_test_838_0_x8_y43_z-25_r0_p-58_y0'.png...
[0.9250919985191978, 1.0, 1.0, 0.9374802838535448, 1.0, 0.9707301383081377, 0.789538431256766, 0.9297977871041587, 1.0, 1.0, 1.0]
680 von 781


[0.9214920092208171, 1.0, 1.0, 0.9388523584078555, 1.0, 0.9854009994323, 0.9271814397188344, 0.9123376623376623, 1.0, 1.0, 1.0]
707 von 781
Neighborhood_test_926_4_x-86_y43_z-27_r0_p-32_y0.png
running deeplab on image b'Neighborhood_test_926_4_x-86_y43_z-27_r0_p-32_y0'.png...
[0.8970133531012553, 1.0, 1.0, 0.9162233741539715, 1.0, 0.9351038582927605, 0.9044579107359753, 0.8155710118914068, 1.0, 1.0, 1.0]
708 von 781
Neighborhood_test_394_4_x-84_y47_z-32_r0_p-43_y0.png
running deeplab on image b'Neighborhood_test_394_4_x-84_y47_z-32_r0_p-43_y0'.png...
[0.9014478411462254, 1.0, 1.0, 0.9182641348200127, 1.0, 0.9492166862540028, 0.85283797492363, 0.7552917903066271, 1.0, 1.0, 1.0]
709 von 781
Neighborhood_test_536_0_x-105_y-41_z-32_r0_p-41_y0.png
running deeplab on image b'Neighborhood_test_536_0_x-105_y-41_z-32_r0_p-41_y0'.png...
[0.8144227822438058, 1.0, 1.0, 0.8857314323714985, 1.0, 0.8359550008615809, 0.9505281771829519, 0.8007774538386784, 1.0, 1.0, 1.0]
710 von 781
Neighborhood_test_

[0.9203353968509439, 1.0, 1.0, 0.9372972242249459, 1.0, 0.9441797470191308, 0.9028116806798662, 0.8731743852203221, 1.0, 1.0, 1.0]
736 von 781
Neighborhood_test_7_4_x-8_y13_z-11_r0_p-16_y0.png
running deeplab on image b'Neighborhood_test_7_4_x-8_y13_z-11_r0_p-16_y0'.png...
[0.952960934189255, 1.0, 1.0, 0.9574479510271713, 1.0, 0.9489132454021921, 0.9517999895501332, 0.9581088362068966, 0.9634522896120665, 1.0, 1.0]
737 von 781
Neighborhood_test_823_0_x-109_y40_z-40_r0_p-47_y0.png
running deeplab on image b'Neighborhood_test_823_0_x-109_y40_z-40_r0_p-47_y0'.png...
[0.9077169418695782, 1.0, 1.0, 0.9208037868601846, 1.0, 0.9292125772641763, 0.8570074397495189, 0.8491996630160067, 1.0, 1.0, 1.0]
738 von 781
Neighborhood_test_601_0_x-97_y57_z-31_r0_p-31_y0.png
running deeplab on image b'Neighborhood_test_601_0_x-97_y57_z-31_r0_p-31_y0'.png...
[0.8832769485903814, 1.0, 1.0, 0.9129829475588457, 1.0, 0.8368097541313986, 0.8952302533223546, 0.8134328358208955, 1.0, 1.0, 1.0]
739 von 781
Neighbo

In [5]:



pathImages = "G:/aeroscapes/SegmentationClassRaw"

class_scores = []       
iou_list = []
i = 0
train = []
iterS = 0
filetrn = open("G:/aeroscapes/ImageSets/val.txt","r")
pathRaw = "G:/result_aeroscapes/vis/raw_segmentation_results"
for l in filetrn:
    l =l.replace("\n","")
    train.append(l)


path = os.listdir(pathImages)
pathR = os.listdir(pathRaw)
lengthR = len(pathR)
for index in tqdm_notebook(range (lengthR)):
    for k in pathR:
        if i < len(train):
            #k= re.sub("_image","_prediction",k)
            print(k)
            original_im = Image.open(os.path.join(pathImages, train[i] + ".png"))
            seg_raw = Image.open(os.path.join(pathRaw,k))

            print('running deeplab on image %s...' % k)
            arr_original = np.array(original_im)
            arr_raw = np.array(seg_raw)
            flat_pred = arr_original.flatten()
            flat_label = arr_raw.flatten()

            iou =compute_mean_iou(flat_pred, flat_label)
            iou_list.append(iou)
            avg_iou = np.mean(iou_list, axis=0)

            print (avg_iou)

print("Average per class validation accuracies for epoch # %04d:"% (epoch))
for index, item in enumerate(class_avg_scores):
    print("%s = %f" % (class_names_list[index], item))




FileNotFoundError: [Errno 2] No such file or directory: 'G:/aeroscapes/ImageSets/val.txt'